In [42]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr  3 03:22:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [43]:
%%writefile coeff_ma.cu

#include <stdio.h>
#include <stdlib.h>

/** AR FUNCTIONS **/
__global__
void autoregressive(size_t n, float *lagged, float *in, int lagged_cols)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index

    int rowStride = blockDim.y * gridDim.y;
    int colStride = blockDim.x * gridDim.x;

    for (int i = row; i < n; i+= rowStride){
      for (int j = col; j < lagged_cols; j+= colStride){
        if (j == 0){
          lagged[i * lagged_cols + j] = 1;
        }else if (i < n && j < lagged_cols) {
            int index = i - j;
            if (index < 0) {
                lagged[i * lagged_cols + j] = 0; // Assign zero for out-of-bounds indices
            } else {
                lagged[i * lagged_cols + j] = in[index]; // Assign lagged value
            }
        }
      }
    }

}

__global__
void transpose(float *out, float *in, int p, size_t ARRAY_SIZE){

  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  if (row < p && col < ARRAY_SIZE) {
      out[row * ARRAY_SIZE + col] = in[col * p + row];
  }

}

__global__
void matMulNaive(float *dest, float *in1, float *in2,
                 size_t in1_height, size_t in2_height,
                 size_t in1_width, size_t in2_width)
{

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Each thread computes one element of the result matrix
    float cValue = 0;

    if (row < in1_height && col < in2_width) {
        // Matrix multiplication: in1 (lagged_cols x n) * in2 (n x lagged_cols)
        for (int k = 0; k < in1_width; ++k) {
            cValue += in1[row * in1_width + k] * in2[k * in2_width + col];
        }
        dest[row * in2_width + col] = cValue;
    }

    //p+1 n x n p+1 -> first mul = p+1 mat
    //p+1 p+1 x p+1 n -> second mul = p+1 x n
}


//Matrix inverse functions
__global__ void nodiag_normalize(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n)
	if (x == i && x!=y){
		I[x*n + y] /= A[i*n + i];
		A[x*n + y] /= A[i*n + i];
	}

}

__global__ void diag_normalize(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n)
	if (x == y && x == i){
		I[x*n + y] /= A[i*n + i];
		A[x*n + y] /= A[i*n + i];
	}

}

__global__ void gaussjordan(float *A, float *I, int n, int i)
{
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n){
		if (x != i){
			I[x*n + y] -= I[i*n + y] * A[x*n + i];
			if (y != i){
				A[x*n + y] -= A[i*n + y] * A[x*n + i];
			}
		}
	}

}

__global__ void set_zero(float *A, float *I, int n, int i){
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;

	if (x < n && y < n){
		if (x != i){
			if (y == i){
				A[x*n + y] = 0;
			}
		}
	}
}

/** MA functions **/

__global__
void calcuate_rate(size_t n, float *out, float *in){
    int k;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (k = index; k < n; k += stride){
      if(k<1)
          out[k] = 0.0f;
      else
       out[k]= in[k]/in[k-1]-1;
    }
}


//output is sum , averaging is done externally
__global__
void  getTotalSum(size_t n, float *avesum ,float *in){
  __shared__ float sum;
    int k;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    sum = 0.0;

    for (k = index; k < n; k +=  stride){
      atomicAdd(&sum, in[k]);
      __syncthreads();

      if(index % blockDim.x ==0){
        atomicAdd(avesum,sum);
      }
    }

}

__global__
void  calculate_residuals(size_t n, float *out ,float *in, float *average ){
    int k;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (k = index; k < n; k += stride){
      if(k<1)
          out[k] = 0.0f;
      else
          out[index] = in[index]- *average;
    }
}

__global__
void calres(float *arrout, float *resids, float *coeff,float *laggedresids, int numLags, int n)
{

  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  int rowStride = blockDim.y * gridDim.y;
  int colStride = blockDim.x * gridDim.x;
  float constant = coeff[0]; // constant is always the first element in the coefficients
  float sum = 0;
  float final = 0.0;

  __shared__ float tempSums;

    for (int i = row; i < n; i+= rowStride){
        tempSums = 0.0f;
      for (int j = col; j < numLags; j+= colStride){
        if (i < n && j < numLags && j != 0) {
            sum= laggedresids[i * numLags + j] *coeff[j];   // sumprod of coeff and laggedresids
            atomicAdd(&tempSums,sum);
        }
        final = constant + resids[i * numLags + j] + tempSums;  // residuals calculation

        if(j+1>=numLags) //store the final answer
          atomicAdd(&arrout[i],final);

        __syncthreads();
      }
    }
}


int main(){

  //dataset
  const size_t ARRAY_SIZE = 100;
  const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

  const int p = 4;
  const int lagged_cols = p + 1;
  const int q = 4;
  const int lagged_q = q+1;


  //ma parameters
  const size_t MA_SIZE = q+1;
  const size_t MA_BYTES = MA_SIZE * sizeof(float);
  const size_t TWOD_BYTES = ARRAY_SIZE * lagged_q * sizeof(float);

  //ma arrays
  float *in, *out, *rate, *residuals_first, *laggingResids_first,
  *residuals_next, *laggingResids_next, *ma_coeff, *totalSum, *residaverage;
  cudaMallocManaged(&residaverage,sizeof(float));
  cudaMallocManaged(&totalSum,sizeof(float));
  cudaMallocManaged(&ma_coeff,MA_BYTES);
  cudaMallocManaged(&in,ARRAY_BYTES);
  cudaMallocManaged(&out,ARRAY_BYTES);
  cudaMallocManaged(&rate,ARRAY_BYTES);
  cudaMallocManaged(&residuals_first,ARRAY_BYTES);
  cudaMallocManaged(&laggingResids_first,TWOD_BYTES);
  cudaMallocManaged(&residuals_next,ARRAY_BYTES);
  cudaMallocManaged(&laggingResids_next,TWOD_BYTES);

  //ar PART
  const size_t AR_SIZE = p + 1;
  const size_t AR_BYTES = AR_SIZE * sizeof(float);
  const size_t X_BYTES = ARRAY_SIZE * lagged_q * sizeof(float);
  const size_t PXP_BYTES = MA_SIZE * MA_SIZE * sizeof(float);


  // declare arrays
  float *lagged, *transposed, *transposed2,
        *prod1, *inverse, *identity, *prod2, *AR_COEFF;
  cudaMallocManaged(&in, ARRAY_BYTES);
  cudaMallocManaged(&out, ARRAY_BYTES);
  cudaMallocManaged(&lagged, X_BYTES); //same amount of rows and p cols
  cudaMallocManaged(&transposed,X_BYTES);
  cudaMallocManaged(&transposed2,X_BYTES);
  cudaMallocManaged(&prod2,X_BYTES);
  cudaMallocManaged(&prod1,PXP_BYTES);
  cudaMallocManaged(&inverse,PXP_BYTES);
  cudaMallocManaged(&identity,PXP_BYTES);
  cudaMallocManaged(&AR_COEFF, AR_BYTES);



  // DEVICE INFO
  int device = -1;
  cudaGetDevice(&device);  //get GPU id


  // memory advise
   cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(in, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(in,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  //"prefetch data" to create GPU page memory
    cudaMemPrefetchAsync(rate,ARRAY_BYTES,device,NULL);


  cudaMemPrefetchAsync(in,ARRAY_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(rate,ARRAY_BYTES,device,NULL);         //"prefetch data" to create GPU page memory

  for (size_t i=0; i<ARRAY_SIZE; i++)
       in[i] = i % 5 + 1.0;

  printf("First 10 elements of input data \n");
  for (int i = 0; i < 10; i++){
    printf("%.5f\n", in[i]);}

  //Displays Last 10 Elements
  printf("Last 10 elements: \n");
  for (int i = ARRAY_SIZE-10; i < ARRAY_SIZE; i++){
    printf("%.5f\n", in[i]);
  }


  //******************FIRST -- CALCULATE RATE
  cudaMemPrefetchAsync(in,ARRAY_BYTES,device,NULL);                                //prefetch from CPU to GPU
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  calcuate_rate<<<numBlocks, numThreads>>>(ARRAY_SIZE,rate,in);

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(rate,ARRAY_SIZE,cudaCpuDeviceId,NULL);

  printf("First 10 elements of rate \n");
  for (int i = 0; i < 10; i++){
    printf("%.5f\n", rate[i]);
  }

   //Displays Last 10 Elements
  printf("Last 10 elements: \n");
  for (int i = ARRAY_SIZE-10; i < ARRAY_SIZE; i++){
    printf("%.5f\n", rate[i]);
  }


  printf("...\n...\n\n");

  //********************SECOND -- GET AVERAGE FOR RESIDUALS
  // memory advise
   cudaMemAdvise(rate, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(rate, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(rate,ARRAY_BYTES,cudaCpuDeviceId,NULL);
  //"prefetch data" to create GPU page memory
    cudaMemPrefetchAsync(totalSum,sizeof(float),device,NULL);

  getTotalSum<<<numBlocks, numThreads>>>(ARRAY_SIZE, totalSum,rate);

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(totalSum,sizeof(float),cudaCpuDeviceId,NULL);

  float average = *totalSum/(ARRAY_SIZE-1);
  *residaverage = average;
  printf("The average is %.5f\n\n", average);

  //********************THIRD -- FIRST RESIDUALS CALCULATION
  // memory advise
   cudaMemAdvise(rate, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(rate, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  //"prefetch data" to create CPU page memory
    cudaMemPrefetchAsync(rate,ARRAY_BYTES,cudaCpuDeviceId,NULL);
    cudaMemPrefetchAsync(residaverage,sizeof(float),cudaCpuDeviceId,NULL);

  //"prefetch data" to create GPU page memory
    cudaMemPrefetchAsync(residuals_first,ARRAY_BYTES,device,NULL);

    calculate_residuals<<<numBlocks, numThreads>>>(ARRAY_SIZE, residuals_first ,rate,residaverage);

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(residuals_first,ARRAY_SIZE,cudaCpuDeviceId,NULL);

  printf("First 10 elements of residuals \n");
  for (int i = 0; i < 10; i++){
    printf("%.5f\n", residuals_first[i]);
  }
  printf("...\n...\n\n");

  //********************FOURTH - CALCULATION OF MA_COEFF && RESIDUALS (LOOPED 5 TIMES)
  // lag and populate the results

  //********** LAG COMPONENTS
  // memory advise
  cudaMemAdvise(residuals_first, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(residuals_first, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(residuals_first,ARRAY_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(laggingResids_first,X_BYTES,device,NULL);         //"prefetch data" to create GPU page memory

  dim3 threadsPerBlock(16, 16);
  //dim3 numBlocks((ARRAY_SIZE + threadsPerBlock.x-1)/threadsPerBlock.x,
  //               (lagged_q + threadsPerBlock.y-1)/threadsPerBlock.y);

  autoregressive<<<numBlocks, threadsPerBlock>>> (ARRAY_SIZE,laggingResids_first, residuals_first, lagged_q);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(laggingResids_first,X_BYTES,cudaCpuDeviceId,NULL);
  printf("Lagged residuals \n");
  for (int i = ARRAY_SIZE-10; i < ARRAY_SIZE; i++){
    for (int j = 0; j < lagged_q; j++){
      printf("%.2f ", laggingResids_first[i*lagged_q+j]);
    }
    printf("\n");
  }



  //********** get first coefficients using autoregressive

  //---------- Transposing lagged matrix ----------------//
  //try removing later along with prev prefetch async and print


  cudaMemAdvise(laggingResids_first,X_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(laggingResids_first,X_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(laggingResids_first,X_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory

  cudaMemPrefetchAsync(transposed,X_BYTES, device, NULL);                                   //"prefetch data" to create GPU page memory

  cudaMemPrefetchAsync(laggingResids_first,X_BYTES, device, NULL);


  //dim3 dimGrid(ARRAY_SIZE/TILE_DIM, p/TILE_DIM, 1);
  //dim3 dimBlock(TILE_DIM, BLOCK_ROWS, 1);

  transpose<<<numBlocks, threadsPerBlock>>>(transposed, laggingResids_first, lagged_q, ARRAY_SIZE);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(transposed,X_BYTES,cudaCpuDeviceId,NULL);


  //---printing tranposed---//
  printf("\n");

  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < 10; j++){
      printf("%.2f ", transposed[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(laggingResids_first, TWOD_BYTES, device, NULL);
  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod1, transposed, laggingResids_first,
                                              lagged_q, ARRAY_SIZE, ARRAY_SIZE, lagged_q);

  cudaDeviceSynchronize();
  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod1, PXP_BYTES, cudaCpuDeviceId, NULL);

  // Print results if needed
  printf("\nMatrix multiplication result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.2f ", prod1[i * lagged_q + j]);
      }
      printf("\n");
  }

  dim3 threadsPerBlockInv(lagged_q, lagged_q);
  dim3 numBlocksInv((lagged_q + lagged_q -1) / lagged_q,
                 (lagged_q+lagged_q-1)/lagged_q);

  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);

  //set identity matrix
  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < lagged_q; j++){
      if (i == j)
        inverse[i * lagged_q + j] = 1.0;
      else
        inverse[i * lagged_q + j] = 0.0;
    }
  }

  printf("\nIdentity Matrix result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.2f ", inverse[i * lagged_q + j]);
      }
      printf("\n");
  }


  for (int i = 0; i < lagged_q; i++){
    nodiag_normalize <<<numBlocksInv, threadsPerBlockInv >>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
		diag_normalize <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
		gaussjordan <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse,lagged_q, i);
    cudaDeviceSynchronize();
		set_zero <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);

  printf("\nInverse result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.9f ", inverse[i * lagged_q + j]);
      }
      printf("\n");
  }

  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(inverse, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod2, inverse, transposed, lagged_q, lagged_q, lagged_q, ARRAY_SIZE);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod2, X_BYTES, cudaCpuDeviceId, NULL);

  printf("\n");
  // Print results if needed
  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < 10; j++){
      printf("%.9f ", prod2[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(ma_coeff, MA_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(ma_coeff, prod2, rate, lagged_q, ARRAY_SIZE, ARRAY_SIZE, 1);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(ma_coeff, MA_BYTES, cudaCpuDeviceId, NULL);


  printf("\n\n----Moving Average coefficients----\n");
  for (int i = 0; i < lagged_q; i++){
    printf("%.5f\n", ma_coeff[i]);
  }

///******************* calculate next residuals

  // memory advise
  cudaMemAdvise(residuals_first, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(residuals_first, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(laggingResids_first, TWOD_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(residuals_first,ARRAY_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(laggingResids_first,TWOD_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(ma_coeff,MA_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory

  cudaMemPrefetchAsync(residuals_next,ARRAY_BYTES,device,NULL);         //"prefetch data" to create GPU page memory

  int threadsPerBlock_in = ARRAY_SIZE/16+1;
  calres<<<threadsPerBlock_in,16>>>(residuals_next,residuals_first,ma_coeff,laggingResids_first,lagged_q,ARRAY_SIZE);
  cudaDeviceSynchronize();  // synchronize GPU with CPU

  cudaMemPrefetchAsync(residuals_next,ARRAY_BYTES,cudaCpuDeviceId,NULL);      // prefetch from GPU to CPU

 //  printf("First ten elements of NEW RESIDUALS\n");
  //for (int i = 0; i < 10; i++){
   // printf("%.5f\n", residuals_next[i]);}

//**************** calculate the next coefficients loop #1

 // memory advise
  cudaMemAdvise(residuals_next, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(residuals_next, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(residuals_next,ARRAY_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(laggingResids_first,TWOD_BYTES,device,NULL);         //"prefetch data" to create GPU page memory



  autoregressive<<<numBlocks, threadsPerBlock>>> (ARRAY_SIZE,laggingResids_first, residuals_next, lagged_q);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(laggingResids_first,TWOD_BYTES,cudaCpuDeviceId,NULL);
  printf("Lagged residuals \n");
  for (int i = ARRAY_SIZE-10; i < ARRAY_SIZE; i++){
    for (int j = 0; j < lagged_q; j++){
      printf("%.2f ", laggingResids_first[i*lagged_q+j]);
    }
    printf("\n");
  }




  //---------- Transposing lagged matrix ----------------//
  //try removing later along with prev prefetch async and print


  cudaMemAdvise(laggingResids_first,TWOD_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(laggingResids_first,TWOD_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);


  cudaMemPrefetchAsync(laggingResids_first,TWOD_BYTES,cudaCpuDeviceId,NULL);                       //"prefetch data" to create CPU page memory

  cudaMemPrefetchAsync(transposed,X_BYTES, device, NULL);                                   //"prefetch data" to create GPU page memory

  cudaMemPrefetchAsync(laggingResids_first,TWOD_BYTES, device, NULL);


  //dim3 dimGrid(ARRAY_SIZE/TILE_DIM, p/TILE_DIM, 1);
  //dim3 dimBlock(TILE_DIM, BLOCK_ROWS, 1);

  transpose<<<numBlocks, threadsPerBlock>>>(transposed, laggingResids_first, lagged_q, ARRAY_SIZE);
  cudaGetLastError();

  // synchronize GPU with CPU
  cudaDeviceSynchronize();

  // prefetch from GPU to CPU
  cudaMemPrefetchAsync(transposed,X_BYTES,cudaCpuDeviceId,NULL);


  //---printing tranposed---//
  printf("\n");

  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < 10; j++){
      printf("%.2f ", transposed[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(laggingResids_first, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod1, transposed, laggingResids_first,
                                              lagged_q, ARRAY_SIZE, ARRAY_SIZE, lagged_q);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod1, PXP_BYTES, cudaCpuDeviceId, NULL);

  // Print results if needed
  printf("\nMatrix multiplication result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.2f ", prod1[i * lagged_q + j]);
      }
      printf("\n");
  }


  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(prod1, PXP_BYTES, device, NULL);

  //set identity matrix
  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < lagged_q; j++){
      if (i == j)
        inverse[i * lagged_q + j] = 1.0;
      else
        inverse[i * lagged_q + j] = 0.0;
    }
  }

  printf("\nIdentity Matrix result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.2f ", inverse[i * lagged_q + j]);
      }
      printf("\n");
  }


  for (int i = 0; i < lagged_q; i++){
    nodiag_normalize <<<numBlocksInv, threadsPerBlockInv >>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
		diag_normalize <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
		gaussjordan <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse,lagged_q, i);
    cudaDeviceSynchronize();
		set_zero <<<numBlocksInv, threadsPerBlockInv>>>(prod1, inverse, lagged_q, i);
    cudaDeviceSynchronize();
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(inverse, PXP_BYTES, cudaCpuDeviceId, NULL);

  printf("\nInverse result:\n");
  for (int i = 0; i < lagged_q; i++) {
      for (int j = 0; j < lagged_q; j++) {
          printf("%.9f ", inverse[i * lagged_q + j]);
      }
      printf("\n");
  }

  cudaMemPrefetchAsync(transposed, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(inverse, PXP_BYTES, device, NULL);
  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(prod2, inverse, transposed, lagged_q, lagged_q, lagged_q, ARRAY_SIZE);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  cudaMemPrefetchAsync(prod2, X_BYTES, cudaCpuDeviceId, NULL);

  printf("\n");
  // Print results if needed
  for (int i = 0; i < lagged_q; i++){
    for (int j = 0; j < 10; j++){
      printf("%.9f ", prod2[i*ARRAY_SIZE+j]);
    }
    printf("\n");
  }

  cudaMemPrefetchAsync(prod2, X_BYTES, device, NULL);
  cudaMemPrefetchAsync(ma_coeff, MA_BYTES, device, NULL);

  matMulNaive<<<numBlocks, threadsPerBlock>>>(ma_coeff, prod2, rate, lagged_q, ARRAY_SIZE, ARRAY_SIZE, 1);

  cudaDeviceSynchronize();
  err = cudaGetLastError();
  if (err != cudaSuccess) {
      printf("CUDA Error: %s\n", cudaGetErrorString(err));
  }

  // Prefetch result back to CPU
  printf("\n\n----Moving Average coefficients II----\n");
  cudaMemPrefetchAsync(ma_coeff, MA_BYTES, cudaCpuDeviceId, NULL);
  for (int i = 0; i < lagged_q; i++){
    printf("%.5f\n", ma_coeff[i]);
  }

  // free ma
  cudaFree(residaverage);
  cudaFree(totalSum);
  cudaFree(ma_coeff);
  cudaFree(in);
  cudaFree(out);
  cudaFree(rate);
  cudaFree(residuals_first);
  cudaFree(laggingResids_first);
  cudaFree(laggingResids_first);
  cudaFree(residuals_next);
  cudaFree(laggingResids_next);

  //free ar  //free memory
  cudaFree(in);
  cudaFree(out);
  cudaFree(lagged);
  cudaFree(transposed);
  cudaFree(prod1);
  cudaFree(prod2);
  cudaFree(inverse);

}


Overwriting coeff_ma.cu


In [44]:
%%shell
nvcc -arch=sm_75 coeff_ma.cu -o coeff_ma

coeff_ma.cu(214): warning #177-D: variable "lagged_cols" was declared but never referenced
    const int lagged_cols = p + 1;
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [45]:
%%shell
nvprof ./coeff_ma

==15583== NVPROF is profiling process 15583, command: ./coeff_ma
First 10 elements of input data 
1.00000
2.00000
3.00000
4.00000
5.00000
1.00000
2.00000
3.00000
4.00000
5.00000
Last 10 elements: 
1.00000
2.00000
3.00000
4.00000
5.00000
1.00000
2.00000
3.00000
4.00000
5.00000
First 10 elements of rate 
0.00000
1.00000
0.50000
0.33333
0.25000
-0.80000
1.00000
0.50000
0.33333
0.25000
Last 10 elements: 
-0.80000
1.00000
0.50000
0.33333
0.25000
-0.80000
1.00000
0.50000
0.33333
0.25000
...
...

The average is 0.26734

First 10 elements of residuals 
0.00000
0.73266
0.23266
0.06599
-0.01734
-1.06734
0.73266
0.23266
0.06599
-0.01734
...
...

Lagged residuals 
1.00 -0.02 0.07 0.23 0.73 
1.00 -1.07 -0.02 0.07 0.23 
1.00 0.73 -1.07 -0.02 0.07 
1.00 0.23 0.73 -1.07 -0.02 
1.00 0.07 0.23 0.73 -1.07 
1.00 -0.02 0.07 0.23 0.73 
1.00 -1.07 -0.02 0.07 0.23 
1.00 0.73 -1.07 -0.02 0.07 
1.00 0.23 0.73 -1.07 -0.02 
1.00 0.07 0.23 0.73 -1.07 

1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
0.00 0.00 0